In [23]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
%pylab inline

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Read the data frame from pickle file

data_dir='../../Data/Weather'
#file_index='BBBSBBBB'
file_index='BBSBSBSB'
from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print 'keys from STAT=',STAT.keys()
#read data
filename=data_dir+'/US_Weather_%s.parquet'%file_index

df=sqlContext.read.parquet(filename)
print df.count()
df.show(5)

Populating the interactive namespace from numpy and matplotlib
keys from STAT= ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP']
12493
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    20|[00 7E 40 D2 00 D...|1898.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    31|[30 D4 80 D4 30 D...|1900.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    28|[00 7E 00 7E A0 D...|1901.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    32|[00 7E 00 7E 00 7...|1902.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    28|[00 00 00 D3 30 D...|1903.0|BBSBSBSB|
+---------+--------+---------+-----------+-----------+------+-------------

In [26]:
yearlydata = []
for
years = df.groupBy('year')

In [30]:
abc = df.dropDuplicates(['year'])

In [31]:
abc.show()

+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    396.8| 47.2467| -94.2228|       TMAX|USC00214652|    31|[00 7E E0 D4 90 D...|1988.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    38|[00 46 00 7E 30 D...|1951.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    43|[80 CD E0 D0 00 7...|1976.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    27|[00 7E 00 7E 00 7...|1940.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    22|[00 7E 00 7E 00 7...|1928.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    31|[30 D4 80 D4 30 D...|1900.0|BBSBSBSB|
|    400.8| 47.4306| -94.0586|       TMAX|USC00219059|    33|[00 7E 00 7E 00 7...|1979.0|BBSBSBSB|
|    400.8

In [32]:
years = abc.select("year").collect()

In [34]:
print STAT_description

[('SortedVals', 'Sample of values', 'vector whose length varies between measurements'), ('UnDef', 'sample of number of undefs per row', 'vector whose length varies between measurements'), ('mean', 'mean value', ()), ('std', 'std', ()), ('low100', 'bottom 1%', ()), ('high100', 'top 1%', ()), ('low1000', 'bottom 0.1%', ()), ('high1000', 'top 0.1%', ()), ('E', 'Sum of values per day', (365,)), ('NE', 'count of values per day', (365,)), ('Mean', 'E/NE', (365,)), ('O', 'Sum of outer products', (365, 365)), ('NO', 'counts for outer products', (365, 365)), ('Cov', 'O/NO', (365, 365)), ('Var', 'The variance per day = diagonal of Cov', (365,)), ('eigval', 'PCA eigen-values', (365,)), ('eigvec', 'PCA eigen-vectors', (365, 365))]


In [36]:
print STAT['TMIN']['eigval']

[  4.39078955e+04   3.38434385e+04   3.27542335e+04  -2.87531637e+04
  -1.80444007e+04   3.00412084e+04   2.88635535e+04   2.81788703e+04
  -1.32356066e+04   2.61188186e+04   2.43922791e+04   2.36552248e+04
   2.29749397e+04   2.19662638e+04  -9.29740827e+03   2.11377751e+04
   2.06054318e+04  -8.64416993e+03   1.93895830e+04   1.86862637e+04
   1.82416024e+04   1.81006470e+04   1.75636457e+04   1.67309682e+04
   1.65924107e+04   1.59683005e+04   1.47300433e+04   1.46199364e+04
   1.43802443e+04   1.41700688e+04   1.37980271e+04   1.34516285e+04
   1.33912145e+04   1.26272039e+04   1.22712397e+04   1.21680509e+04
   1.16750742e+04   1.13370022e+04   1.12276253e+04   1.11620324e+04
   1.00415419e+04   1.07167304e+04   1.02348645e+04   1.05065751e+04
  -6.58047816e+03  -6.40414502e+03  -5.07012403e+03  -4.80862752e+03
   9.88155086e+03   9.46814160e+03   9.30527658e+03   9.12738533e+03
   8.93664957e+03   8.79137688e+03   8.40494854e+03   8.13478819e+03
   8.01389559e+03   7.98643018e+03